In [12]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from skimage.feature import hog
from skimage import exposure
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import pytorch as torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pytorch'

In [ ]:
class_label_encoding = {
    'SPOILED': 0,
    'HALF': 1,
    'FRESH': 2
}

# Loading Images

In [ ]:
def load_images(file_path, output_x, output_y):
    for file_name in os.listdir(file_path):
        class_name = file_name.split('-')[0]
        if (class_name == '_classes.csv'): continue
        img = cv.imread(file_path + file_name).astype('float32')
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, (128, 128), interpolation = cv.INTER_AREA)
        img /= 255
        output_x.append(img)
        output_y.append(class_label_encoding[class_name])

In [ ]:
train_x = []
train_y = []
test_x = []
test_y = []

load_images('data/train/', train_x, train_y)
load_images('data/valid/', test_x, test_y)

In [ ]:
train_x[0]

# Feature Extraction

## Color Histogram
Jason

In [ ]:
def extract_color_histogram(image, bins=32):
    """
    Extract color histogram features from an image.
    
    Parameters:
    - image: The input image (should be in RGB format)
    - bins: Number of bins for the histogram
    
    Returns:
    - histogram_features: Flattened histogram features
    """
    # Extract histograms for each channel
    hist_r = cv.calcHist([image], [0], None, [bins], [0, 1])  # Changed range to [0, 1] since you're normalizing images
    hist_g = cv.calcHist([image], [1], None, [bins], [0, 1])
    hist_b = cv.calcHist([image], [2], None, [bins], [0, 1])
    
    # Normalize the histograms
    cv.normalize(hist_r, hist_r, 0, 1, cv.NORM_MINMAX)
    cv.normalize(hist_g, hist_g, 0, 1, cv.NORM_MINMAX)
    cv.normalize(hist_b, hist_b, 0, 1, cv.NORM_MINMAX)
    
    # Flatten and concatenate the histograms
    histogram_features = np.concatenate([
        hist_r.flatten(), 
        hist_g.flatten(), 
        hist_b.flatten()
    ])
    
    return histogram_features

In [ ]:
def plot_color_histogram(image, bins=32, title="Color Histogram"):
    """
    Plot the color histogram of an image.
    
    Parameters:
    - image: The input image (should be in RGB format)
    - bins: Number of bins for the histogram
    - title: Title for the plot
    
    Returns:
    - None (displays the plot)
    """
    # Create a figure with subplots
    fig, ax = plt.subplots(1, 4, figsize=(16, 4))
    
    # Display the original image
    ax[0].imshow(image)
    ax[0].set_title('Original Image')
    ax[0].axis('off')
    
    # Get histogram features using your existing function
    features = extract_color_histogram(image, bins)
    
    # Split the features back into channels
    channel_length = len(features) // 3
    hist_r = features[:channel_length].reshape(bins, 1)
    hist_g = features[channel_length:2*channel_length].reshape(bins, 1)
    hist_b = features[2*channel_length:].reshape(bins, 1)
    
    # Define colors and channels
    colors = ['r', 'g', 'b']
    channels = ['Red', 'Green', 'Blue']
    hists = [hist_r, hist_g, hist_b]
    
    # Plot histograms for each channel
    for i, (hist, col, chan) in enumerate(zip(hists, colors, channels)):
        ax[i+1].plot(hist, color=col)
        ax[i+1].set_xlim([0, bins])
        ax[i+1].set_title(f'{chan} Histogram')
        ax[i+1].set_xlabel('Bins')
        ax[i+1].set_ylabel('# of Pixels')
        ax[i+1].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return features

In [ ]:
image = train_x[0]  # Get the first image
plot_color_histogram(image, bins=32, title="Meat Sample Color Histogram")
print()

In [ ]:
# Extract features from training and testing sets
train_features = []
for img in train_x:
    hist_features = extract_color_histogram(img)
    train_features.append(hist_features)
train_features = np.array(train_features)


test_features = []
for img in test_x:
    hist_features = extract_color_histogram(img)
    test_features.append(hist_features)
test_features = np.array(test_features)

print(train_features)

## Local Binary Pattern
Aiden

In [ ]:
def get_pixel(img, center, x, y): 
      
    new_value = 0
      
    try: 
        # if local neighbourhood pixel value is greater than or equal to center pixel values then set it to 1 
        if img[x][y] >= center: 
            new_value = 1
              
    except: 
        # exception required when neighbourhood value of center pixel value is null
        pass
      
    return new_value 
   
# Function for calculating LBP 
def lbp_calculated_pixel(img, x, y): 
   
    center = img[x][y] 
   
    val_ar = [] 
      
    # top_left 
    val_ar.append(get_pixel(img, center, x-1, y-1)) 
      
    # top 
    val_ar.append(get_pixel(img, center, x-1, y)) 
      
    # top_right 
    val_ar.append(get_pixel(img, center, x-1, y + 1)) 
      
    # right 
    val_ar.append(get_pixel(img, center, x, y + 1)) 
      
    # bottom_right 
    val_ar.append(get_pixel(img, center, x + 1, y + 1)) 
      
    # bottom 
    val_ar.append(get_pixel(img, center, x + 1, y)) 
      
    # bottom_left 
    val_ar.append(get_pixel(img, center, x + 1, y-1)) 
      
    # left 
    val_ar.append(get_pixel(img, center, x, y-1)) 
       
    # convert binary values to decimal 
    power_val = [1, 2, 4, 8, 16, 32, 64, 128] 
   
    val = 0
      
    for i in range(len(val_ar)): 
        val += val_ar[i] * power_val[i] 
          
    return val


def lbp_output(img_bgr):
    height, width, _ = img_bgr.shape 
   
    # convert RGB to gray 
    img_gray = cv.cvtColor(img_bgr, 
                            cv.COLOR_BGR2GRAY) 
       
    # create numpy array as same height and width of RGB image 
    img_lbp = np.zeros((height, width), 
                       np.float32) 
       
    for i in range(0, height): 
        for j in range(0, width): 
            img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)

    return img_lbp

In [ ]:
img_bgr = train_x[0]
img_lbp = lbp_output(img_bgr)
  
plt.imshow(img_bgr) 
plt.show()
   
plt.imshow(img_lbp, cmap ="gray")
plt.show()

In [ ]:
def save_images_lbp(imgs, labels, train_test='train'):
    label_text = ['SPOILED', 'HALF', 'FRESH']
    for image in range(len(imgs)):
        lbp_image = lbp_output(imgs[image])
        filename = f'data/lbp/{train_test}/{label_text[labels[image]]}-{image}-lbp.jpg'
        cv.imwrite(filename, lbp_image)

In [ ]:
# save_images_lbp(train_x, train_y, train_test='train')

In [ ]:
# save_images_lbp(test_x, test_y, train_test='test')

## Histograms of Oriented Gradients
Fiona

In [ ]:
# Adapted from here: https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_hog.html
def make_hog(image, visualize=False):
    if visualize:
        features, hog_image = hog(
            image,
            orientations=16,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            visualize=visualize,
            channel_axis=-1
        )

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

        ax1.axis('off')
        ax1.imshow(image, cmap=plt.cm.gray)
        ax1.set_title('Input image')

        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

        ax2.axis('off')
        ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
        ax2.set_title('Histogram of Oriented Gradients')
        plt.show()
        return features
    else:
        features = hog(
            image,
            orientations=16,
            pixels_per_cell=(8, 8),
            cells_per_block=(1, 1),
            visualize=visualize,
            channel_axis=-1
        )

In [ ]:
make_hog(train_x[1], True)

In [ ]:
train_features_hog = []
test_features_hog = []

for image in train_x:
    train_features_hog.append(make_hog(image))

for image in test_x:
    test_features_hog.append(make_hog(image))

# Classification

## Decision Tree
Jason

In [ ]:
def train_decision_tree(x_train_tree, y_train_tree, x_test_tree, y_test_tree, max_depth=5, show_tree=True, feature_names=None):
    """
    Train a decision tree classifier on any type of features, with optional histogram visualization.
    
    Parameters:
    - x_train_tree: Training features
    - y_train_tree: Training labels
    - x_test_tree: Test features
    - y_test_tree: Test labels
    - max_depth: Maximum depth of the decision tree
    - show_tree: Whether to visualize the decision tree
    - feature_names: Names of features (will be auto-generated if None)
    
    Returns:
    - dt_classifier: Trained decision tree classifier
    - accuracy: Classification accuracy on test set
    - report: Classification report
    """

    # Create a list of class names
    class_names=['SPOILED', 'HALF', 'FRESH']
    
    # Create and train a Decision Tree classifier
    dt_classifier = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    dt_classifier.fit(x_train_tree, y_train_tree)
    
    # Make predictions
    predictions = dt_classifier.predict(x_test_tree)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test_tree, predictions)
    report = classification_report(y_test_tree, predictions, target_names=class_names)
    
    # Print results
    print(f"Decision Tree Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(report)
    
    # Show decision tree if requested
    if show_tree:
        # Create feature names if not provided
        if feature_names is None:
            feature_names = [f"Feature_{i}" for i in range(x_train_tree.shape[1])]
            
        plt.figure(figsize=(15, 10))
        plot_tree(dt_classifier, 
                  feature_names=feature_names,
                  class_names=class_names,
                  filled=True, 
                  rounded=True, 
                  fontsize=8)
        plt.title("Decision Tree for Classification")
        plt.tight_layout()
        plt.show()
    
    return dt_classifier, accuracy, report

In [ ]:
# Create feature names for the histogram features
bins_per_channel = train_features.shape[1] // 3
channels = ['Red', 'Green', 'Blue']
feature_names = []
for channel in channels:
    for index in range(bins_per_channel):
        feature_names.append(f"{channel} Bin {index}")

#x_train_tree, y_train_tree, x_test_tree, y_test_tree, max_depth=5, show_tree=True, feature_names=None

# Train the decision tree with histogram visualization
model, acc, report = train_decision_tree(x_train_tree=train_features, y_train_tree=train_y, x_test_tree=test_features,
                                         y_test_tree=test_y, max_depth=3, feature_names=feature_names)

## Random Forest
Aiden

In [ ]:
def train_random_forest(x_train_forest, y_train_forest, x_test_forest, y_test_forest, n_estimators=100, criterion='gini', max_depth=None,
                        min_samples_split=2, min_samples_leaf=1, max_features='sqrt'):
    # Create Random Forest classifer object
    clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf, max_features=max_features, random_state=42)
    
    # Train Random Forest Classifer
    clf.fit(x_train_forest,y_train_forest)
    
    #Predict the response for test dataset
    y_pred = clf.predict(x_test_forest)
    
    accuracy = accuracy_score(y_test_forest, y_pred)
    precision = precision_score(y_test_forest, y_pred)
    recall = recall_score(y_test_forest, y_pred)
    f1 = f1_score(y_test_forest, y_pred)
    confusion = confusion_matrix(y_test_forest, y_pred)

    return accuracy, precision, recall, f1

# Neural Network

In [ ]:


class FeedforwardNN(nn.Module):
    """
    A configurable feedforward neural network implemented with PyTorch
    """
    def __init__(self, input_dim, hidden_layers=[64, 32], output_classes=3, 
                 dropout_rate=0.2, activation='relu', batch_norm=True):
        """
        Initialize the neural network.
        
        Parameters:
        -----------
        input_dim : int
            Number of input features
        hidden_layers : list of int
            List containing the number of neurons in each hidden layer
        output_classes : int
            Number of output classes (3 for SPOILED, HALF, FRESH)
        dropout_rate : float
            Dropout rate for regularization
        activation : str
            Activation function ('relu', 'sigmoid', 'tanh')
        batch_norm : bool
            Whether to use batch normalization
        """
        super(FeedforwardNN, self).__init__()
        
        # Choose activation function
        if activation == 'relu':
            self.activation_fn = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation_fn = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation_fn = nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")
        
        # Build network architecture
        layers = []
        prev_dim = input_dim
        
        # Add hidden layers
        for i, units in enumerate(hidden_layers):
            # Add linear layer
            layers.append(nn.Linear(prev_dim, units))
            
            # Add batch normalization if enabled
            if batch_norm:
                layers.append(nn.BatchNorm1d(units))
            
            # Add activation function
            layers.append(self.activation_fn)
            
            # Add dropout if enabled
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            
            prev_dim = units
        
        # Add output layer
        layers.append(nn.Linear(prev_dim, output_classes))
        
        # Create sequential model
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        """Forward pass through the network."""
        return self.model(x)


class MeatClassifier:
    """
    A wrapper class for training and evaluating the PyTorch neural network model.
    """
    def __init__(self, input_dim, hidden_layers=[64, 32], output_classes=3, 
                 dropout_rate=0.2, activation='relu', batch_norm=True,
                 learning_rate=0.001, weight_decay=0.001, device=None):
        """
        Initialize the classifier.
        
        Parameters:
        -----------
        input_dim : int
            Number of input features
        hidden_layers : list of int
            List containing the number of neurons in each hidden layer
        output_classes : int
            Number of output classes (3 for SPOILED, HALF, FRESH)
        dropout_rate : float
            Dropout rate for regularization
        activation : str
            Activation function ('relu', 'sigmoid', 'tanh')
        batch_norm : bool
            Whether to use batch normalization
        learning_rate : float
            Learning rate for optimizer
        weight_decay : float
            Weight decay (L2 regularization) strength
        device : str or None
            Device to use ('cuda' or 'cpu'). If None, will use CUDA if available.
        """
        # Set device
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(device)
        
        # Initialize model
        self.model = FeedforwardNN(
            input_dim=input_dim,
            hidden_layers=hidden_layers,
            output_classes=output_classes,
            dropout_rate=dropout_rate,
            activation=activation,
            batch_norm=batch_norm
        ).to(self.device)
        
        # Initialize optimizer
        self.optimizer = optim.Adam(
            self.model.parameters(), 
            lr=learning_rate, 
            weight_decay=weight_decay
        )
        
        # Initialize loss function (CrossEntropyLoss includes softmax)
        self.criterion = nn.CrossEntropyLoss()
        
        # For tracking training progress
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        
        # For storing the best model
        self.best_val_loss = float('inf')
        self.best_model_state = None
        
        # For feature scaling
        self.scaler = StandardScaler()
    
    def train(self, features, labels, test_size=0.2, batch_size=32, epochs=100, 
              early_stopping_patience=20, random_state=42, verbose=True):
        """
        Train the neural network.
        
        Parameters:
        -----------
        features : numpy array
            Feature matrix
        labels : numpy array
            Labels
        test_size : float
            Proportion of data to use for validation
        batch_size : int
            Batch size for training
        epochs : int
            Number of training epochs
        early_stopping_patience : int
            Number of epochs with no improvement after which training will stop
        random_state : int
            Random seed for reproducibility
        verbose : bool
            Whether to print training progress
            
        Returns:
        --------
        self : MeatClassifier
            The trained classifier
        """
        # Convert to numpy arrays if not already
        features = np.array(features)
        labels = np.array(labels)
        
        # Split data into train and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            features, labels, test_size=test_size, 
            random_state=random_state, stratify=labels
        )
        
        # Scale features
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(self.device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(self.device)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(self.device)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(self.device)
        
        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        
        # For early stopping
        patience_counter = 0
        
        # Training loop
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            correct_train = 0
            total_train = 0
            
            for inputs, targets in train_loader:
                # Forward pass
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                
                # Backward pass and optimize
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                # Track statistics
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_train += targets.size(0)
                correct_train += (predicted == targets).sum().item()
            
            # Calculate training metrics
            epoch_train_loss = train_loss / len(train_loader)
            epoch_train_acc = correct_train / total_train
            self.train_losses.append(epoch_train_loss)
            self.train_accuracies.append(epoch_train_acc)
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0
            
            with torch.no_grad():
                for inputs, targets in val_loader:
                    # Forward pass
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, targets)
                    
                    # Track statistics
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += targets.size(0)
                    correct_val += (predicted == targets).sum().item()
            
            # Calculate validation metrics
            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_acc = correct_val / total_val
            self.val_losses.append(epoch_val_loss)
            self.val_accuracies.append(epoch_val_acc)
            
            # Save the best model
            if epoch_val_loss < self.best_val_loss:
                self.best_val_loss = epoch_val_loss
                self.best_model_state = self.model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Print progress
            if verbose and (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}: "
                      f"train_loss={epoch_train_loss:.4f}, "
                      f"train_acc={epoch_train_acc:.4f}, "
                      f"val_loss={epoch_val_loss:.4f}, "
                      f"val_acc={epoch_val_acc:.4f}")
            
            # Early stopping
            if early_stopping_patience > 0 and patience_counter >= early_stopping_patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch+1}")
                break
        
        # Load the best model
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
        
        return self
    
    def predict(self, features):
        """
        Make predictions on new data.
        
        Parameters:
        -----------
        features : numpy array
            Feature matrix
            
        Returns:
        --------
        numpy array
            Predicted class indices
        """
        # Scale features
        features_scaled = self.scaler.transform(features)
        
        # Convert to PyTorch tensor
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(self.device)
        
        # Make predictions
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            _, predictions = torch.max(outputs, 1)
        
        return predictions.cpu().numpy()
    
    def predict_proba(self, features):
        """
        Predict class probabilities.
        
        Parameters:
        -----------
        features : numpy array
            Feature matrix
            
        Returns:
        --------
        numpy array
            Class probabilities
        """
        # Scale features
        features_scaled = self.scaler.transform(features)
        
        # Convert to PyTorch tensor
        X_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(self.device)
        
        # Make predictions
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probabilities = torch.softmax(outputs, dim=1)
        
        return probabilities.cpu().numpy()
    
    def evaluate(self, features, labels):
        """
        Evaluate the model on test data.
        
        Parameters:
        -----------
        features : numpy array
            Feature matrix
        labels : numpy array
            True labels
            
        Returns:
        --------
        dict
            Dictionary containing evaluation metrics
        """
        # Get predictions
        y_pred = self.predict(features)
        
        # Calculate accuracy
        accuracy = accuracy_score(labels, y_pred)
        
        # Generate classification report
        class_names = ['SPOILED', 'HALF', 'FRESH']
        report = classification_report(labels, y_pred, target_names=class_names, output_dict=True)
        
        return {
            'accuracy': accuracy,
            'report': report
        }
    
    def plot_training_history(self):
        """Plot the training history (loss and accuracy curves)."""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Loss plot
        ax1.plot(self.train_losses, label='Training Loss')
        ax1.plot(self.val_losses, label='Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title('Loss Curves')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Accuracy plot
        ax2.plot(self.train_accuracies, label='Training Accuracy')
        ax2.plot(self.val_accuracies, label='Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.set_title('Accuracy Curves')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def save_model(self, filepath):
        """Save the model to a file."""
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scaler': self.scaler,
            'best_val_loss': self.best_val_loss
        }, filepath)
    
    def load_model(self, filepath):
        """Load the model from a file."""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scaler = checkpoint['scaler']
        self.best_val_loss = checkpoint['best_val_loss']
        
        return self


def train_meat_classifier_pytorch(features, labels, hidden_layers=[128, 64], 
                                  learning_rate=0.001, weight_decay=0.001,
                                  batch_size=32, epochs=100, test_size=0.2,
                                  early_stopping_patience=20, dropout_rate=0.2,
                                  activation='relu', batch_norm=True,
                                  random_state=42, verbose=True):
    """
    Train and evaluate a neural network for meat classification using PyTorch.
    
    Parameters:
    -----------
    features : numpy array
        Feature matrix (color histogram, HOG, or LBP features)
    labels : numpy array
        Labels (0: SPOILED, 1: HALF, 2: FRESH)
    hidden_layers : list of int
        List of hidden layer sizes
    learning_rate : float
        Learning rate for optimizer
    weight_decay : float
        Weight decay (L2 regularization) strength
    batch_size : int
        Batch size for training
    epochs : int
        Maximum number of epochs for training
    test_size : float
        Proportion of data to use for testing
    early_stopping_patience : int
        Number of epochs with no improvement after which training will stop
    dropout_rate : float
        Dropout rate for regularization
    activation : str
        Activation function ('relu', 'sigmoid', 'tanh')
    batch_norm : bool
        Whether to use batch normalization
    random_state : int
        Random seed for reproducibility
    verbose : bool
        Whether to print training progress
        
    Returns:
    --------
    dict
        Dictionary containing the classifier, evaluation metrics, and other info
    """
    # Create classifier
    classifier = MeatClassifier(
        input_dim=features.shape[1],
        hidden_layers=hidden_layers,
        output_classes=len(np.unique(labels)),
        dropout_rate=dropout_rate,
        activation=activation,
        batch_norm=batch_norm,
        learning_rate=learning_rate,
        weight_decay=weight_decay
    )
    
    # Train the classifier
    classifier.train(
        features=features,
        labels=labels,
        test_size=test_size,
        batch_size=batch_size,
        epochs=epochs,
        early_stopping_patience=early_stopping_patience,
        random_state=random_state,
        verbose=verbose
    )
    
    # Split data for evaluation (using the same random state for consistency)
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=test_size, 
        random_state=random_state, stratify=labels
    )
    
    # Evaluate on test set
    evaluation = classifier.evaluate(X_test, y_test)
    
    # Plot training history
    classifier.plot_training_history()
    
    # Return results
    return {
        'classifier': classifier,
        'accuracy': evaluation['accuracy'],
        'report': evaluation['report']
    }


# Example usage:
"""
# For color histogram features
color_hist_results = train_meat_classifier_pytorch(
    features=np.array(train_features),  # Assuming this is your color histogram features
    labels=np.array(train_y),
    hidden_layers=[128, 64, 32],
    epochs=150,
    batch_size=16
)

# Print accuracy and classification report
print(f"Test accuracy: {color_hist_results['accuracy']:.4f}")
print("\nClassification Report:")
for class_name, metrics in color_hist_results['report'].items():
    if isinstance(metrics, dict):  # Skip aggregated metrics
        print(f"{class_name}: Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}, F1-score={metrics['f1-score']:.2f}")

# For HOG features
hog_results = train_meat_classifier_pytorch(
    features=np.array(train_features_hog),  # Assuming this is your HOG features
    labels=np.array(train_y),
    hidden_layers=[64, 32],
    epochs=150,
    weight_decay=0.01
)
"""